# JS02 - Key Value Store

Macrometa GDN is a geo-distributed real-time coordination-free materialized views engine that supports multiple data models. You can use GDN as a geo-replicated real-time key-value datastore or database.

If you are new to Macrometa GDN, start by reading the essentials of Macrometa GDN.

Each document stored in a collection (or table) contains a primary key, `_key`. The rest of the document is considered a value. If it has no secondary indexes, the collection behaves like a simple key-value (KV) store.

The key-value store has no query languages. The permissible operations are key look-ups (single and batch), key-value pair insertions, updates, and deletions. If you don't specify a sharding attribute, we use `_key` to shard the data. The simplicity of this model makes a key-value store fast, easy to use, scalable, portable, and flexible.

You can enable `time_to_live` (TTL) during collection creation and add an expireAt value to specify the expiration time for each document in the KV collection.

For the following examples, assume these credentials:

## Pre-requisite

Let's assume: 
- You have already made a tenant account and have a username and password
- You have installed the jsC8 drivers as explained in section 01
- You have generated an API Key as explained in section 01


In [ ]:
#/* run this once to install javascript kernal and jsc8 in google colab, then reload, and then skip this
!npm install jsc8
!npm install -g --unsafe-perm ijavascript
!ijsinstall --install=global  # */

## Step 02-A - Connect to GDN

The first step in using GDN is to connect to a local region. When this code executes, it initializes the server connection to the region URL you specified. You can create an API key from the GUI or REST API.

In [ ]:
let emailId, password;
let fed_url = "https://gdn.paas.macrometa.io";

In [ ]:
$$.input(
  { prompt: "Please, type fedration url, leave blank to use default and press enter: " },
  (error, url) => {
      if(url)
        fed_url = url;
    $$.done();
  }
);

In [ ]:
$$.input(
  { prompt: "Please, type your email and press enter: " },
  (error, email) => {
    emailId = email;
    $$.done();
  }
);

In [ ]:
$$.input(
  { prompt: "Please, type your password and press enter: ", password: true },
  (error, pass) => {
    password = pass;
    $$.done();
  }
);

In [ ]:
const jsc8 = require("jsc8");

// ----- simple way  -----
const client = new jsc8({
  url: fed_url,
});

client
  .login(emailId, password)
  .then((result) => console.log("Login successfully", result))
  .catch((err) => console.error("Error while login", err.message));

// ----- with token -----
/* const client = new jsc8({
  url: "https://gdn.paas.macrometa.io",
  token: "XXXX",
  fabricName: "_system",
}); */

// ----- with apikey-----
/* const client = new jsc8({
  url: "https://gdn.paas.macrometa.io",
  apiKey: "XXXX",
  fabricName: "_system",
}); */

## Step 02-B - Create a Collection

In [ ]:
const collection_name = "students";

const createCollection = async () => {
  let collectionDetails;
  try {
    const isCollectionExists = await client.hasCollection(collection_name);
    if (isCollectionExists) {
      console.log("Collection exists with name ", collection_name);
      return;
    }
    collectionDetails = await client.createKVCollection(collection_name);
    console.log("Collection created! ", collection_name);
  } catch (err) {
    console.error("Collection creation did not succeed due to ", err.message);
  }
};

createCollection();

## Step 02-C - Insert Key Value Pairs

Insert key-value pairs into the collection:

In [ ]:
let data = [
  {
    _key: "John",
    value: "Science",
    expireAt: 0,
  },
  {
    _key: "Alice",
    value: "Maths",
    expireAt: 0,
  },
  {
    _key: "Alex",
    value: "Physics",
    expireAt: 0,
  },
  {
    _key: "Monika",
    value: "Chemistry",
    expireAt: 0,
  },
];

const insertKVPairs = async () => {
  try {
    await client.insertKVPairs(collection_name, data);
    console.log("KV pairs inserted");
  } catch (err) {
    console.error("Error while inserting KV pairs", err.message);
  }
};

insertKVPairs();

## Step 02-D - Get a Value

Get a value for a given key:

In [ ]:
const getValueForKey = async () => {
  try {
    const monikaKV = await client.getValueForKey(collection_name, "Monika");
    console.log("Value for the provided key Monika: ", monikaKV);

    const johnKV = await client.getValueForKey(collection_name, "John");

    console.log("Value for the provided key John: ", johnKV);
  } catch (err) {
    console.error("Error while reading the KV", err.message);
  }
};

getValueForKey();

## Step 02-E - Get Key-Value Count

Get the key-value count from a given collection:

In [ ]:
const getKVCount = async () => {
  try {
    const result = await client.getKVCount(collection_name);
    console.log("Number of kv pairs in your collection: ", result.count);
  } catch (err) {
    console.error("Error while fetching kv count ", err.message);
  }
};

getKVCount();

## Step 02-F - Update Value

Update a value for a given key:

In [ ]:
data = {
  _key: "John",
  value: "Biology",
  expireAt: 0,
};

const insertKeyValuePair = async () => {
  try {
    await client.insertKVPairs(collection_name, data);
    console.log("Updated the specified KV pair");
  } catch (err) {
    console.error("Error while updating KV pair", err.message);
  }
};

insertKeyValuePair();

## Step 02-G - Delete a Key-Value

Delete key-value pairs from a collection:

In [ ]:
const deleteEntryForKey = async () => {
  try {
    await client.deleteEntryForKey(collection_name, "John");
    console.log("Deleted Entry for the specified Key John");
  } catch (err) {
    console.error("Error while deleting KV pair", err.message);
  }
};

deleteEntryForKey();

## Step 02-H - Delete a Key-Value

Delete an entry for multiple key-value pairs from a collection:

In [ ]:
const deleteEntryForKeys = async () => {
  try {
    const values2 = await client.deleteEntryForKeys(collection_name, [
      "Monika",
      "Alex",
      "Alice",
    ]);
    console.log("Deleted Entries for the specified Key: ", values2);
  } catch (err) {
    console.error("Error while deleting KV pair", err.message);
  }
};

deleteEntryForKeys();

## Step 02-I - Delete a Collection

Delete the key-value collection:

In [ ]:
const deleteKVCollection = async () => {
  try {
    await client.deleteKVCollection(collection_name);
    console.log("Collection Deleted: ", collection_name);
  } catch (err) {
    console.error("Error while deleting collection ", err.message);
  }
};

deleteKVCollection();

## Workbook Completed!

Congratulations! You should have successfully created a collection, inserted values, updated a value, deleted an entry, deleted all entries, and finally deleted the collection. 

Don't forget you can log into your GDN account and check the console to see what is happening from the dashboard at the same time!